# Sentence Similarity

In [1]:
!pip install -U sentence-transformers &> /dev/null

In [3]:
import pandas as pd
queries = pd.read_excel('/content/queries.xlsx')
queries = queries['Prompt']
queries = list(queries)

In [54]:
import re
with open('/content/fish.txt') as f:
    text = f.read()

sentences = re.split(r'\n', text)

In [55]:
ar1 = pd.DataFrame(sentences, columns=['text'])
ar1 = ar1[ar1['text'].str.len()>0]
len(ar1)

18

In [ ]:
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [56]:
query = queries[0]
sentences = list(ar1.text)

#Compute embedding for both lists
embedding_1= model.encode(query, convert_to_tensor=True)
embedding_2 = model.encode(sentences, convert_to_tensor=True)

similarity = util.pytorch_cos_sim(embedding_1, embedding_2).numpy()[0]

In [57]:
ar1['query'] = query
ar1['similarity'] = similarity
ar1 = ar1.sort_values(by='similarity', ascending=False)
ar1

,text,query,similarity
15,affect the volume of catch misreported (the to...,Vessel caught misreporting catch amount,0.672065
2,The report further states that around US$152m ...,Vessel caught misreporting catch amount,0.630068
4,Components of the unreported catch may include...,Vessel caught misreporting catch amount,0.599801
9,Given the large number of juveniles caught whe...,Vessel caught misreporting catch amount,0.598432
3,A commercial fisherman has had his $450000 ves...,Vessel caught misreporting catch amount,0.543626
7,Both misreporting and not reporting of catch d...,Vessel caught misreporting catch amount,0.539028
17,"Sources could include late catch reporting, mi...",Vessel caught misreporting catch amount,0.526358
0,The company also faces costs of £2396 and must...,Vessel caught misreporting catch amount,0.509688
13,Operators may under-declare catches for partic...,Vessel caught misreporting catch amount,0.493958
10,For catching vessels fishing actively for blue...,Vessel caught misreporting catch amount,0.484003


In [58]:
ar1.head(10).similarity.mean()

0.5597025

In [59]:
query = 'Vessel caught with unregistered catch'

In [60]:
import re
with open('/content/fish1.txt') as f:
    text = f.read()

sentences = re.split(r'\n', text)
ar2 = pd.DataFrame(sentences, columns=['text'])
ar2 = ar2[ar2['text'].str.len()>0]
len(ar2)

13

In [61]:
sentences2 = list(ar2.text)

#Compute embedding for both lists
embedding_1= model.encode(query, convert_to_tensor=True)
embedding_2 = model.encode(sentences2, convert_to_tensor=True)

similarity = util.pytorch_cos_sim(embedding_1, embedding_2).numpy()[0]

In [62]:
ar2['query'] = query
ar2['similarity'] = similarity
ar2 = ar2.sort_values(by='similarity', ascending=False)
ar2

,text,query,similarity
5,A significant proportion of illegal catch is m...,Vessel caught with unregistered catch,0.595708
11,The penalty for operating an unregistered vess...,Vessel caught with unregistered catch,0.578175
7,Unlawfully catching and retaining any fish.,Vessel caught with unregistered catch,0.534469
1,"If you steal a ship, it's unregistered.",Vessel caught with unregistered catch,0.498240
8,Unreported fishing refers to fishing activitie...,Vessel caught with unregistered catch,0.480473
9,Illegal fishing takes place when vessels or ha...,Vessel caught with unregistered catch,0.462616
0,An unregistered ship is stateless and breaches...,Vessel caught with unregistered catch,0.440503
2,Vessels including the Russian trawler Novaya Z...,Vessel caught with unregistered catch,0.414068
3,records that the master of the vessel shall ke...,Vessel caught with unregistered catch,0.403380
12,It is an offence to operate a vessel in a way ...,Vessel caught with unregistered catch,0.401712


In [64]:
ar2.similarity.mean()

0.44639963

# Text relevance

In [66]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-v2-m3')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-v2-m3')
model.eval()

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tensor([-8.1838,  5.2650])


In [70]:
query = 'Vessel caught misreporting catch amount'
output = ['It was escorted by the coast guard, under arrest for illegally fishing in Argentinian waters.',
          'Today’s high seas are a bit like the Old West.',
          'Arrested for misreporting fish',
          'Vessel caught misreporting catch amount']
pairs = [[query, output[0]], [query, output[1]], [query, output[2]], [query, output[3]]]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

tensor([ -7.4526, -11.0387,  -0.3019,  10.3176])


# News classification using NER

In [73]:
import spacy
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [75]:
text="""The South Korean-flagged trawler belonged to the fleet operated by the Sajo Oyang
corporation, notorious for its record of high seas transgressions, as documented by The Guardian.
In recent years, the Oyang 77 had gotten in trouble in New Zealand for illegally dumping
dead fish overboard, underreporting catch and failing to pay workers, according to a report from
Oceana, a nonprofit focused on ocean conservancy. In February 2019, the Argentine Coast Guard
discovered the trawler with its nets extended inside the EEZ. They found more than 310,000 pounds
of seafood on board. Leaving nothing to chance, they deployed a helicopter and an airplane
to assist the Coast Guard in escorting the Oyang 77 to shore, releasing it after confiscating
its fishing equipment and extracting a fine of 25 million Argentine pesos, or about $550,000."""

doc = nlp(text)

for ent in doc.ents:
    print(ent.text, "|",ent.label_, "|", spacy.explain(ent.label_))

South Korean | NORP | Nationalities or religious or political groups
Sajo Oyang | PERSON | People, including fictional
Guardian | ORG | Companies, agencies, institutions, etc.
recent years | DATE | Absolute or relative dates or periods
Oyang | ORG | Companies, agencies, institutions, etc.
New Zealand | GPE | Countries, cities, states
Oceana | ORG | Companies, agencies, institutions, etc.
February 2019 | DATE | Absolute or relative dates or periods
the Argentine Coast Guard | ORG | Companies, agencies, institutions, etc.
EEZ | ORG | Companies, agencies, institutions, etc.
more than 310,000 pounds | QUANTITY | Measurements, as of weight or distance
the Coast Guard | ORG | Companies, agencies, institutions, etc.
Oyang | ORG | Companies, agencies, institutions, etc.
25 million | CARDINAL | Numerals that do not fall under another type
Argentine | NORP | Nationalities or religious or political groups
about $550,000 | MONEY | Monetary values, including unit
